# SCALETTA
  ### 1. Import delle librerie


### INTRODUZIONE AL PROGETTO (== abstract)
 1) Motivazioni
 2) Spiegazione degli approcci utilizzati
 3) Accenno ai risultati ottenuti

Insipration for the project came from this [paper](https://arxiv.org/pdf/1703.07579.pdf) from 2017, using some unupdated approaches.

# 0. SETUP

In [ ]:
import os
import random
import math
import time as tm
from collections import deque
import gdown
import tarfile
import json
import numpy as np
import cv2
import pandas as pd
from enum import Enum
from functools import total_ordering
from PIL import Image #, ImageDraw

import clip

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler, SequentialSampler
from torch.distributions import Categorical
import torchvision
from torchvision import transforms

from ultralytics import YOLO

# TODO: VERIFICARE CORRETTO FUNZIONAMENTO!!!!
import argparse
import wandb
# fro env.env import *

### Global variables definition

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

HISTORY_LENGHT = 12
ACT_NUM = 9
INPUT_SIZE = 1024+1024+5+ACT_NUM*HISTORY_LENGTH
BBOX_EMB=1024
NUM_SEGMENTS=9


VALIDATION_SET_SIZE=2572

# x. Problem Statement

## x.1 Description

## x.2 Dataset and Dataloader (TODO: update dataset to work with dataloader)

Our dataset is **RefCOCOg**, a dataset for referring expression comprehension in the context of visual grounding, based on the **COCO** dataset. We created a custom *PyTorch* class to deal with it, downloading and/or extracting it if necessary.

In [ ]:
class RefCOCOg(Dataset):
    FILE_ID = "1wyyksgdLwnRMC9pQ-vjJnNUn47nWhyMD"
    ARCHIVE_NAME = "refcocog.tar.gz"
    NAME = "refcocog"
    ANNOTATIONS = "annotations/refs(umd).p"
    JSON = "annotations/instances.json"
    IMAGES = "images"
    IMAGE_NAME = "COCO_train2014_{}.jpg"
    FUSION_NET_PATH = "fusion.pt"


    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self._check_dataset()
        self._filter_annotation(
            os.path.join(self.data_dir, self.NAME, self.ANNOTATIONS)
        )
        self._load_json()
        self.transform = transform
        self.model, self.preprocess = clip.load("RN50", device=DEVICE)#,jit=False)
        # ret = torch.load("fusion.pt",map_location=DEVICE)
        # self.fusion_embedding_net = FusionEmbedding()
        # self.fusion_embedding_net.load_state_dict(torch.load(RefCOCOg.FUSION_NET_PATH)["model"])
        # self.fusion_embedding_net.eval()
        # checkpoint = torch.load("../RN-50-REFCOCOG.pt")
        
        # Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
        # checkpoint['model_state_dict']["input_resolution"] = self.model.input_resolution #default is 224
        # checkpoint['model_state_dict']["context_length"] = self.model.context_length # default is 77
        # checkpoint['model_state_dict']["vocab_size"] = self.model.vocab_size 

        # self.model.load_state_dict(checkpoint['model_state_dict'])
        self.index_list_train = [i for i in range(0,len(self.annotation_train))]
        self.index_list_val = [i for i in range(0,len(self.annotation_val))]
        random.shuffle(self.index_list_train)
        random.shuffle(self.index_list_val)


    def _check_dataset(self):
        if not os.path.exists(os.path.join(self.data_dir, self.ARCHIVE_NAME)):
            if not os.path.exists(self.data_dir):
                os.mkdir(self.data_dir)
            print("Downloading dataset...")
            gdown.download(id=self.FILE_ID)
        if not os.path.exists(os.path.join(self.data_dir, self.NAME)):
            print("Extracting dataset...")
            with tarfile.open(
                os.path.join(self.data_dir, self.ARCHIVE_NAME), "r:gz"
            ) as tar:
                tar.extractall(path=self.data_dir)
        else:
            print("Dataset already extracted")


    def _load_json(self):
        with open(os.path.join(self.data_dir, self.NAME, self.JSON)) as f:
            self.json = json.load(f)
        self.json = pd.DataFrame(self.json["annotations"])


    def __len__(self):
        return len(self.annotation)


    def __getitem__(self, item):
        idx,split = item
        if split is None:
            split = "train"
        # get the random index from shuffled list
        # get line by index
        if split == "train":
            random_index = self.index_list_train[idx]
            raw = self.annotation_train.iloc[random_index]
        elif split == "val":
            random_index = self.index_list_val[idx]
            raw = self.annotation_val.iloc[random_index]
        else:
            raise ValueError("split must be train or val!")
        # get image
        image = self._get_image(raw)
        # get sentences
        
        sentences = self._get_sentences(raw)
        # get bbox

        bboxes = self._get_bboxes(raw)

        # return self._get_vector bboxes and image width and height
        return self._get_vector(image, sentences) , bboxes, image.width, image.height, image, sentences


    def _get_image(self, raw):
        # get image_id
        image_id = raw["image_id"]
        # pad image_id to 12 digits
        image_id = str(image_id).zfill(12)
        # convert image to tensor
        image = Image.open(
            os.path.join(
                self.data_dir, self.NAME, self.IMAGES, self.IMAGE_NAME.format(image_id)
            )
        )
        return image


    def _get_sentences(self, raw):
        # get sentences
        sentences = raw["sentences"]
        # get raw sentences
        sentences = [sentence["raw"] for sentence in sentences]
        return sentences


    def _get_bboxes(self, raw):
        # get ref_id
        id = raw["ann_id"]
        bboxes = self.json[self.json["id"] == id]["bbox"].values[0]
        return bboxes


    def _filter_annotation(self, path):
        self.annotation = pd.read_pickle(path)
        self.annotation = pd.DataFrame(self.annotation)
        self.annotation_train = self.annotation[self.annotation["split"] == "train"]
        self.annotation_val = self.annotation[self.annotation["split"] == "val"]


    def _get_vector(self, image, sentences):
            image = self.preprocess(image).unsqueeze(0).to(DEVICE)
            for i in range(0,len(sentences)):
                text = "a photo of "+sentences[i]
            text = clip.tokenize(sentences).to(DEVICE)
            with torch.no_grad():
                image_features = self.model.encode_image(image)
                text_features = self.model.encode_text(text)
            text_features = torch.mean(text_features,dim=0).to(DEVICE)
            # text_features = text_features.to(DEVICE).squeeze(0)
            # image_features = image_features.to(DEVICE).squeeze(0)
            # out = torch.cat((image_features, text_features),dim=0).to(DEVICE)
            
            # Combine image and text features and normalize
            product = torch.mul(image_features, text_features)
            power = torch.sign(product)* torch.sqrt(torch.abs(product))
            out = torch.div(power, torch.norm(power, dim=1).reshape(-1, 1))
            out =torch.mean(out,dim=0)

            # print(out.shape, power_out.shape)
            # append bbox
            # print(f"Output shape: {power_out.shape}")
            
            # out = torch.cat((image_features, text_selected),dim=1).to(DEVICE).unsqueeze(0)
            # return self.fusion_embedding_net(out).squeeze(0).squeeze(0).to(DEVICE)    
            return out

In [ ]:
train_dataset = RefCOCOg(".", "train")
val_dataset = RefCOCOg(".", "val")
test_dataset = RefCOCOg(".", "test")

The same goes for the dataloader. Our batch size is set to *one*, because an agent processes one image at a time. The only way to process mutliple images at once would be to increase the number of agent, which would add so much overhead to the implementation just for reducing the training time, without improvement on the agent behavior, since the policy is updated after **5k** steps, and introducing a problem of managing the history of the parallel agents' actions. 

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# 1. ENVIRONMENT CREATION

Enum class used for actions rappresentation

In [ ]:
@total_ordering
class Actions(Enum):
    ACT_RT = 0 #Right
    ACT_LT = 1 #Left
    ACT_UP = 2 #Up
    ACT_DN = 3 #Down
    ACT_TA = 4 #Taller
    ACT_FA = 5 #Fatter
    ACT_SR = 6 #Shorter
    ACT_TH = 7 #Thiner
    ACT_TR = 8 #Trigger 


    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value

## 1.1 OpenAI Gym

When dealing with a reinforcement learning problem, we first need to define the environment where the agent will learn. We decided to use [OpenAI Gym](https://gym.openai.com/), a toolkit that provides convinient APIs to simplify our trainging process.

First, we define our **observation** and **action** spaces: 

- **Observation space**: as described above, the agent state contains information about image, bounding box, bounding box image and history, with float values in a $R^1$ space.

- **Action space**: as already mentioned, the agent has to choose one action among 9 possible actions, making it a discrete action space. 

And then, we have to define a **reset** and **step** function:

- **Reset**: this function is called at the beginning of each episode, and it returns the initial state of the agent. In our case, based on the setup choosen, the initial state is representend by:

    - the YOLO bounding box with the highest **cosine similarity** with the provided sentences.

    - the full image bounding box

- **Step**: this function is called at each step of the episode, and it returns the next state, the reward and a boolean value that indicates if the episode is over or not. In our case, after choosing the action, the agent will move the bounding box accordingly, compute the new **intersection over union** and use it to compute the new reward. The episode ends when the action choosed is ***TRIGGER*** or the agent has moved the bounding box 

    - **12 times** when starting from YOLO best bounding box
    
    - **50 times** when starting from the full image bounding box

Eventually, we can define the **render** function, that will be used to visualize the agent's behaviour.

TODO: esaustiva spiegazione del funzionamento dell'environment

In [ ]:

class VisualGroundingEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}
    CONVERGENCE_THRESHOLD = 0.6


    def __init__(self,dataset, num_agent=1,agent_offset=0,render_mode=None,random_validation=True):
        # self.window_size = 512  # The size of the PyGame window
        self.dataset = dataset
        self.idx = {"train": 0, "val": 0}
        self.iou = 0
        self.current_iou=0
        self.highest_iou=0
        self.avg_similarity=0
        self.num_agent=num_agent
        self.agent_offset=agent_offset
        self.idx["train"]= self.agent_offset
        self.idx["val"]= self.agent_offset
        self.max_steps_per_episode = 12
        # self._max_episode_steps=50
        self.steps_num=0
        self.trigger_pressed=False
        # wheter to extract images in random order or not 
        self.random_validation=random_validation
        self.YOLO = YOLO("yolov8n.pt")
        self.observation_space = spaces.Box(low=-100000, high=100000, shape=(INPUT_SIZE,))
        self.action_history = deque(maxlen = HISTORY_LENGTH)
        # We have 9 actions, corresponding to "right", "up", "left", "down", "v-shrink", "v-stretch", "h-shrink", "h-stretch", "confirm"
        self.action_space = spaces.Discrete(ACT_NUM)
        print("Env initialized")


    def _get_info(self, trigger_pressed):
        return {"pred_bbox": self._agent_location, "target_bbox": self._target_location, "trigger_pressed":trigger_pressed, "img_idx":self.idx[self.split]}


    def compute_vbbox(self):
      area_bbox = self.bbox_width * self.bbox_height
      area_img = self.width * self.height
      v_bbox = torch.tensor( [self.x1/(self.width-1), self.y1/(self.height-1), self.x2/(self.width-1), self.y2/(self.height-1), area_bbox/area_img])
      return v_bbox


    def history2vec(self):
      """
      Convert action history deque to vector for constructing state
      :param q: deque contains histories
      """
      history = np.array(self.action_history)
      res = np.zeros((HISTORY_LENGTH, ACT_NUM))
      if len(history) != 0:
        res[np.arange(len(history)) + (HISTORY_LENGTH - len(history)), history] = 1
      res = res.reshape((HISTORY_LENGTH * ACT_NUM)).astype(np.float32)
      return res
    
    
    def select_best_segment_CLIP(self,image,sentences,n):
      width, height = image.size[0], image.size[1]
      sentences = clip.tokenize(sentences).to(DEVICE)
      similarities = []
      img_embeddings = []
      coordinates = []
      imgs=[]
      with torch.no_grad():

        # Wrapper function used to create the embedding
        def create_embedding(image, coordinates):
          image_crop = image.crop(coordinates)
          imgs.append(image_crop)
          image_emb = self.dataset.preprocess(image_crop).unsqueeze(0).to(DEVICE)
          image_emb = self.dataset.model.encode_image(image_emb)
          return image_emb

        sentences_emb = self.dataset.model.encode_text(sentences).to(DEVICE)
        # print(sentences_emb.shape)
        sentences_emb = torch.mean(sentences_emb, dim=0,dtype=float).unsqueeze(0).to(DEVICE)

        for i in range(n):
          for j in range(n):
            x1,y1,x2,y2 = j*width//n, i*height//n, (j+1)*width//n , (i+1)*height//n
            coordinates.append((x1,y1,x2,y2))
            img_embeddings.append(create_embedding(image, (x1,y1,x2,y2)))
            # compute similarity between image and sentence
            similarities.append(torch.cosine_similarity(img_embeddings[-1], sentences_emb,dim=-1).item())
            if i+1 < n:
                y2_new = (i+2)*height//n
                coordinates.append((x1,y1,x2,y2_new))
                img_embeddings.append(create_embedding(image,(x1,y1,x2,y2_new)))
                similarities.append(torch.cosine_similarity(img_embeddings[-1], sentences_emb,dim=-1).item())
            if j+1 < n:
                x2_new = (j+2)*width//n
                coordinates.append((x1,y1,x2_new,y2))
                img_embeddings.append(create_embedding(image,(x1,y1,x2_new,y2)))
                similarities.append(torch.cosine_similarity(img_embeddings[-1], sentences_emb,dim=-1).item())
            if i+1 < n and j+1 < n:
                y2_new = (i+2)*height//n
                x2_new = (j+2)*width//n
                coordinates.append((x1,y1,x2_new,y2_new))
                img_embeddings.append(create_embedding(image,(x1,y1,x2_new,y2_new)))
                similarities.append(torch.cosine_similarity(img_embeddings[-1], sentences_emb,dim=-1).item())
        #retrieve the segment with the highest similarity
        return img_embeddings[np.argmax(similarities)].squeeze(), coordinates[np.argmax(similarities)]


    def select_best_segment_YOLO(self,img,sentences):

      # Wrapper function used to create the embedding
      def create_embedding(image, coordinates):
          image_crop = image.crop(coordinates)
          image_emb = self.dataset.preprocess(image_crop).unsqueeze(0).to(DEVICE)
          image_emb = self.dataset.model.encode_image(image_emb)
          return image_emb,image_crop

      full_img_emb = self.dataset.preprocess(img).unsqueeze(0).to(DEVICE)
      # print("\n\n\nSENTENCES: ",sentences)
      text_emb = clip.tokenize(sentences).to(DEVICE)
      with torch.no_grad():
          full_img_emb = self.dataset.model.encode_image(full_img_emb)
          text_emb = self.dataset.model.encode_text(text_emb)
      # text_emb = text_emb[0]

      text_emb = torch.mean(text_emb,dim=0,dtype=torch.float).to(DEVICE)
      similarities=[]
      img_embs=[]
      coordinates=[]
      # imgs=[]
      #YOLO PREDICTIONS
      cv2_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
      # cv2.imshow("full image",cv2_img)
      results = self.YOLO.predict(cv2_img,verbose=False)
      for r in results:
          # annotator = Annotator(cv2_img)
          boxes = r.boxes
          for box in boxes:
            # get box coordinates in (top, left, bottom, right) format
              x1,y1,x2,y2 = box.xyxy[0]
              pred_emb,_ = create_embedding(img,(int(x1),int(y1),int(x2),int(y2)))
              coordinates.append((x1.item(),y1.item(),x2.item(),y2.item()))
              img_embs.append(pred_emb)
              # imgs.append(cv2_img[int(y1):int(y2) , int(x1):int(x2)])
              # label = box.cls
              # annotator.box_label((x1,y1,x2,y2), YOLO_.names[int(label)]+str(len(crops)-1))
              similarities.append(torch.cosine_similarity(pred_emb, text_emb,dim=-1).item())
              # img_result = annotator.result()
              # NO PREDICTIONS -> RETURN FULL IMAGE
      # cv2.imshow("YOLO",imgs[np.argmax(similarities)])
      # cv2.waitKey(1)
      # tm.sleep(4)
      if len(similarities) == 0:
        return full_img_emb.squeeze(), (0,0,img.width,img.height)
      else:
        return img_embs[np.argmax(similarities)].squeeze(), coordinates[np.argmax(similarities)]


    def reset(self, seed=None, options=None):
        # print("GIOU : ",self.current_iou,"\t\t num steps: ",self.steps_num)
        # if self.trigger_pressed:
        #     print("TRIGGER")
        self.split = "train"
        if options is not None:
           self.split = options["split"]
        # print("GIOU : ",str(round(100*self.current_iou,3)),"% with ",self.steps_num) #| AVG_SIMILARITY: ",str(round(100*self.avg_similarity,3))+"%")
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        
        embeddings, bbox, width, height, image, sentences = self.dataset[self.idx[self.split],self.split]
        self.sentences = sentences
        self.image=image
        self.img_txt_emb = embeddings
        # dividing image into NUM_SEGMENTS segments (j*k lines) and choosing the one with the highest similarity
        self.bbox_emb, (self.x1,self.y1,self.x2,self.y2) = self.select_best_segment_YOLO(self.image, self.sentences)#,int(math.sqrt(NUM_SEGMENTS)))
        # print("Image has shape ",self.image.size," and bbox_emb agent is in ", (self.x1,self.y1,self.x2,self.y2))
        self.width = width
        self.height = height
        # Choose the agent's location uniformly
        # torch.set_printoptions(threshold=10_000)
        self.bbox_width = self.x2 - self.x1
        self.bbox_height = self.y2 - self.y1
        self._agent_location = torch.tensor([[self.x1, self.y1, self.x2, self.y2]]).to(DEVICE)

        v_bbox = self.compute_vbbox().to(DEVICE)
        bbox_x2 = bbox[0]+bbox[2]
        bbox_y2 = bbox[1]+bbox[3]
        self._target_location = torch.tensor([[bbox[0],bbox[1],bbox_x2,bbox_y2 ]]).to(DEVICE)
        self.current_iou = torchvision.ops.distance_box_iou( self._agent_location ,self._target_location )[0].item()
        self.iou = self.current_iou
        self.current_max_iou = self.iou
        self.highest_iou = self.current_iou
        self.action_history = deque(maxlen=HISTORY_LENGTH)
        
        v_hist = torch.zeros(ACT_NUM*HISTORY_LENGTH).to(DEVICE)
        state = torch.cat( (self.img_txt_emb.to(DEVICE),self.bbox_emb,v_bbox,v_hist) ).to(DEVICE)
        self.steps_num = 0
        # self.idx = self.idx + 1 
        info = self._get_info(False)
        self.idx[self.split] +=self.num_agent
        
        # reset dataset counter
        if self.split=="train" and (self.idx[self.split] >= len(self.dataset.annotation_train)-1):
          random.shuffle(self.dataset.index_list_train)
          self.idx[self.split] = self.agent_offset

        if self.split=="val" and self.random_validation:
           self.idx[self.split] = random.randint(self.agent_offset,len(self.dataset.annotation_val)-1)
        state =state.detach().cpu().numpy()
        return state.squeeze(), info


    def _update_bbox(self, action):
      ALPHA = 0.2
      BETA  = 0.1
      self.x2 = self.x1 + self.bbox_width
      self.y2 = self.y1 + self.bbox_height

      assert action >= Actions.ACT_RT.value and action <= Actions.ACT_TR.value
      if action <= Actions.ACT_DN.value:
        delta_w = int(ALPHA * self.bbox_width)
        delta_h = int(ALPHA * self.bbox_height)
      else:
        delta_w = int(BETA * self.bbox_width)
        delta_h = int(BETA * self.bbox_height)
      # PREVENT_STUCK:
      if (delta_h == 0):
        delta_h = 2
      if (delta_w == 0):
        delta_w = 2

      if action == Actions.ACT_RT.value:
        self.x1 += delta_w
        self.x2 += delta_w
      elif action == Actions.ACT_LT.value:
        self.x1 -= delta_w
        self.x2 -= delta_w
      elif action == Actions.ACT_UP.value:
        self.y1 -= delta_h
        self.y2 -= delta_h
      elif action == Actions.ACT_DN.value:
        self.y1 += delta_h
        self.y2 += delta_h
      elif action == Actions.ACT_TA.value:
        self.y1 -= delta_h
        self.y2 += delta_h
      elif action == Actions.ACT_FA.value:
        self.x1 -= delta_w
        self.x2 += delta_w
      elif action == Actions.ACT_SR.value:
        self.y1 += delta_h
        self.y2 -= delta_h
      elif action == Actions.ACT_TH.value:
        self.x1 += delta_w
        self.x2 -= delta_w
      elif action == Actions.ACT_TR.value:
        pass
      else:
        raise ValueError('Invalid action')
      # ensure bbox inside image
      if self.x1 < 0:
        self.x1 = 0
      if self.y1 < 0:
        self.y1 = 0
      if self.x2 < 0:
        self.x2 = 0
      if self.y2 < 0:
        self.y2 = 0
      if self.x1 >= self.width:
        self.x1 = self.width - 1
      if self.y1 >= self.height:
        self.y1 = self.height - 1
      if self.x2 >= self.width:
        self.x2 = self.width - 1
      if self.y2 >= self.height:
        self.y2 = self.height - 1
      # assert self.x1 < self.x2
      # assert self.y1 < self.y2

      # Ensure bbox doesn't get flipped
      x1 = min(self.x1, self.x2)
      y1 = min(self.y1, self.y2)
      x2 = max(self.x1, self.x2)
      y2 = max(self.y1, self.y2)
      if x1 ==x2:
          x2+=1
      if y1 ==y2:
          y2+=1

      self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2

      return


    def step(self, action):
        self.steps_num+=1
        done = False
        reward = 0
        self.iou = self.current_iou

        self._update_bbox(action)
        # An episode is done iff the agent has reached the target

        self.action_history.append(action)
        v_hist = torch.from_numpy(self.history2vec()).to(DEVICE)
        # print("action: ",action," -> current action hist: ",self.action_history)
        # print("Step: current vhist: ",v_hist)
        # if self.render_mode == "human":
        #     self._render_frame()
        self.bbox_width = self.x2 - self.x1
        self.bbox_height = self.y2 - self.y1
        self._agent_location =  torch.tensor([[self.x1, self.y1, self.x2, self.y2]]).to(DEVICE)
        v_bbox = self.compute_vbbox().to(DEVICE)
        
        agent_bbox_crop = self.image.crop( (self.x1, self.y1, self.x2, self.y2) )
        img = self.dataset.preprocess(agent_bbox_crop).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            self.bbox_emb = self.dataset.model.encode_image(img).squeeze(0).to(DEVICE)
        #state = torch.cat( (self.img_txt_emb.to(DEVICE),self.bbox_emb,v_bbox) ).to(DEVICE)
        
        state = torch.cat( (self.img_txt_emb.to(DEVICE),self.bbox_emb,v_bbox,v_hist) ).to(DEVICE)

        self.current_iou = torchvision.ops.distance_box_iou( self._agent_location, self._target_location )[0].item()

        # self.avg_similarity = torch.mean(bbox_similarity).item()
        self.trigger_pressed = False
        # if self.current_iou > VisualGroundingEnv.CONVERGENCE_THRESHOLD or self.steps_num > 49 or action==Actions.ACT_TR.value:
        if self.steps_num >= self.max_steps_per_episode or action==Actions.ACT_TR.value:
            done = True
            if action==Actions.ACT_TR.value:
                self.trigger_pressed = True
            # if (self.current_iou > VisualGroundingEnv.CONVERGENCE_THRESHOLD):
            #   print(f"IOU% > {VisualGroundingEnv.CONVERGENCE_THRESHOLD*100}% ---> tot = {self.done_counter}")

        reward += self._calculate_reward(self.iou,self.current_iou,action==Actions.ACT_TR.value)
        info = self._get_info(self.trigger_pressed)
        state = state.detach().cpu().numpy()
        return state, reward, done, info


    def _calculate_reward(self, previous_iou, current_iou, is_stop_action):
      if is_stop_action:
        if current_iou > VisualGroundingEnv.CONVERGENCE_THRESHOLD:
          return 0.50  # Reward for correctly stopping when IoU has improved
        else:
          return -0.50  # Penalty for stopping when IoU has not improved
      else:
        iou_difference = current_iou - previous_iou
        if iou_difference > 0:
          return iou_difference  # Reward for an action that increases IoU
        else:
          return iou_difference  # Penalty for an action that decreases IoU

        
    def _calculate_reward(self, previous_iou, current_iou, is_stop_action):
      #Get reward
      if not is_stop_action:
        if (current_iou > self.current_max_iou):
          reward = current_iou
          self.current_max_iou = current_iou
        else:
          reward = -0.1
      else:
        iou = torchvision.ops.box_iou( self._agent_location, self._target_location )[0].item()
        if iou < VisualGroundingEnv.CONVERGENCE_THRESHOLD:
          reward = -0.75
        else:
          reward = 0.75
      reward += (-previous_iou + 0.98 * current_iou)
      return reward


    def close(self):
        pass



# 2. UTILITY FUNCTIONS DEFINITION

### Functions from 'normalization.py'

In [ ]:
class RunningMeanStd:
    # Dynamically calculate mean and std
    def __init__(self, shape):  # shape:the dimension of input data
        self.n = 0
        self.mean = np.zeros(shape)
        self.S = np.zeros(shape)
        self.std = np.sqrt(self.S)


    def update(self, x):
        x = np.array(x)
        self.n += 1
        if self.n == 1:
            self.mean = x
            self.std = x
        else:
            old_mean = self.mean.copy()
            self.mean = old_mean + (x - old_mean) / self.n
            self.S = self.S + (x - old_mean) * (x - self.mean)
            self.std = np.sqrt(self.S / self.n)

In [ ]:
class Normalization:
    def __init__(self, shape):
        self.running_ms = RunningMeanStd(shape=shape)


    def __call__(self, x, update=True):
        # Whether to update the mean and std,during the evaluating,update=False
        if update:
            self.running_ms.update(x)
        x = (x - self.running_ms.mean) / (self.running_ms.std + 1e-8)

        return x

In [ ]:

class RewardScaling:
    def __init__(self, shape, gamma):
        self.shape = shape  # reward shape=1
        self.gamma = gamma  # discount factor
        self.running_ms = RunningMeanStd(shape=self.shape)
        self.R = np.zeros(self.shape)


    def __call__(self, x):
        self.R = self.gamma * self.R + x
        self.running_ms.update(self.R)
        x = x / (self.running_ms.std + 1e-8)  # Only divided std
        return x


    def reset(self):  # When an episode is done,we should reset 'self.R'
        self.R = np.zeros(self.shape)

### Functions and classes from 'utils.py' (I removed dataset)

In [ ]:
class FusionEmbedding(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(2048, 1024).to(DEVICE)
    self.fc2 = nn.Linear(1024, 1024).to(DEVICE)
    self.gelu = nn.GELU().to(DEVICE)
    self.batch_norm = nn.BatchNorm1d(1024).to(DEVICE)


  def forward(self, embedding):
    # embedding.float32().to(DEVICE)
    embedding = embedding.to(torch.float32).to(DEVICE)
    fc1_out = self.fc1(embedding)
    gelu_out = self.gelu(fc1_out)
    gelu_out = gelu_out.permute(0,2,1)
    norm_out = self.batch_norm(gelu_out)
    norm_out = norm_out.permute(0,2,1)
    fc2_out = self.fc2(norm_out)
    out = self.gelu(fc2_out)
    return out

In [ ]:
def denormalize(img,x0_norm,y0_norm,x1_norm,y1_norm):
    width = img.shape[1]
    height = img.shape[0]
    # print("den",width,height,sep=" | ")
    x0 = int(x0_norm * width)
    y0 = int(y0_norm * height)
    x1 = int(x1_norm * width)
    y1 = int(y1_norm * height)
    return x0,y0,x1,y1

In [ ]:
def draw_bbox(img,x0,y0,x1,y1):
    img = cv2.rectangle(img, (x0, y0), (x1, y1), (255,0,0), 2)
    return img

In [ ]:
def compute_iou(agent, ground_truth):
    iou =  torchvision.ops.box_iou( agent, ground_truth)[0].item()
    print("iou : ",iou)
    return iou

In [ ]:
class RefCOCOg(Dataset):
    FILE_ID = "1wyyksgdLwnRMC9pQ-vjJnNUn47nWhyMD"
    ARCHIVE_NAME = "refcocog.tar.gz"
    NAME = "refcocog"
    ANNOTATIONS = "annotations/refs(umd).p"
    JSON = "annotations/instances.json"
    IMAGES = "images"
    IMAGE_NAME = "COCO_train2014_{}.jpg"
    FUSION_NET_PATH = "fusion.pt"


    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self._check_dataset()
        self._filter_annotation(
            os.path.join(self.data_dir, self.NAME, self.ANNOTATIONS)
        )
        self._load_json()
        self.transform = transform
        self.model, self.preprocess = clip.load("RN50", device=DEVICE)#,jit=False)
        # ret = torch.load("fusion.pt",map_location=DEVICE)
        # self.fusion_embedding_net = FusionEmbedding()
        # self.fusion_embedding_net.load_state_dict(torch.load(RefCOCOg.FUSION_NET_PATH)["model"])
        # self.fusion_embedding_net.eval()
        # checkpoint = torch.load("../RN-50-REFCOCOG.pt")
        
        # Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
        # checkpoint['model_state_dict']["input_resolution"] = self.model.input_resolution #default is 224
        # checkpoint['model_state_dict']["context_length"] = self.model.context_length # default is 77
        # checkpoint['model_state_dict']["vocab_size"] = self.model.vocab_size 

        # self.model.load_state_dict(checkpoint['model_state_dict'])
        self.index_list_train = [i for i in range(0,len(self.annotation_train))]
        self.index_list_val = [i for i in range(0,len(self.annotation_val))]
        random.shuffle(self.index_list_train)
        random.shuffle(self.index_list_val)


    def _check_dataset(self):
        if not os.path.exists(os.path.join(self.data_dir, self.ARCHIVE_NAME)):
            if not os.path.exists(self.data_dir):
                os.mkdir(self.data_dir)
            print("Downloading dataset...")
            gdown.download(id=self.FILE_ID)
        if not os.path.exists(os.path.join(self.data_dir, self.NAME)):
            print("Extracting dataset...")
            with tarfile.open(
                os.path.join(self.data_dir, self.ARCHIVE_NAME), "r:gz"
            ) as tar:
                tar.extractall(path=self.data_dir)
        else:
            print("Dataset already extracted")


    def _load_json(self):
        with open(os.path.join(self.data_dir, self.NAME, self.JSON)) as f:
            self.json = json.load(f)
        self.json = pd.DataFrame(self.json["annotations"])


    def __len__(self):
        return len(self.annotation)


    def __getitem__(self, item):
        idx,split = item
        if split is None:
            split = "train"
        # get the random index from shuffled list
        # get line by index
        if split == "train":
            random_index = self.index_list_train[idx]
            raw = self.annotation_train.iloc[random_index]
        elif split == "val":
            random_index = self.index_list_val[idx]
            raw = self.annotation_val.iloc[random_index]
        else:
            raise ValueError("split must be train or val!")
        # get image
        image = self._get_image(raw)
        # get sentences
        
        sentences = self._get_sentences(raw)
        # get bbox

        bboxes = self._get_bboxes(raw)

        # return self._get_vector bboxes and image width and height
        return self._get_vector(image, sentences) , bboxes, image.width, image.height, image, sentences


    def _get_image(self, raw):
        # get image_id
        image_id = raw["image_id"]
        # pad image_id to 12 digits
        image_id = str(image_id).zfill(12)
        # convert image to tensor
        image = Image.open(
            os.path.join(
                self.data_dir, self.NAME, self.IMAGES, self.IMAGE_NAME.format(image_id)
            )
        )
        return image


    def _get_sentences(self, raw):
        # get sentences
        sentences = raw["sentences"]
        # get raw sentences
        sentences = [sentence["raw"] for sentence in sentences]
        return sentences


    def _get_bboxes(self, raw):
        # get ref_id
        id = raw["ann_id"]
        bboxes = self.json[self.json["id"] == id]["bbox"].values[0]
        return bboxes


    def _filter_annotation(self, path):
        self.annotation = pd.read_pickle(path)
        self.annotation = pd.DataFrame(self.annotation)
        self.annotation_train = self.annotation[self.annotation["split"] == "train"]
        self.annotation_val = self.annotation[self.annotation["split"] == "val"]


    def _get_vector(self, image, sentences):
            image = self.preprocess(image).unsqueeze(0).to(DEVICE)
            for i in range(0,len(sentences)):
                text = "a photo of "+sentences[i]
            text = clip.tokenize(sentences).to(DEVICE)
            with torch.no_grad():
                image_features = self.model.encode_image(image)
                text_features = self.model.encode_text(text)
            text_features = torch.mean(text_features,dim=0).to(DEVICE)
            # text_features = text_features.to(DEVICE).squeeze(0)
            # image_features = image_features.to(DEVICE).squeeze(0)
            # out = torch.cat((image_features, text_features),dim=0).to(DEVICE)
            
            # Combine image and text features and normalize
            product = torch.mul(image_features, text_features)
            power = torch.sign(product)* torch.sqrt(torch.abs(product))
            out = torch.div(power, torch.norm(power, dim=1).reshape(-1, 1))
            out =torch.mean(out,dim=0)

            # print(out.shape, power_out.shape)
            # append bbox
            # print(f"Output shape: {power_out.shape}")
            
            # out = torch.cat((image_features, text_selected),dim=1).to(DEVICE).unsqueeze(0)
            # return self.fusion_embedding_net(out).squeeze(0).squeeze(0).to(DEVICE)    
            return out

### ReplayBuffer class

In [ ]:
class ReplayBuffer:
    def __init__(self, args):
        self.gamma = args.gamma
        self.lamda = args.lamda
        self.use_adv_norm = args.use_adv_norm
        self.state_dim = args.state_dim
        self.action_dim = args.action_dim
        self.episode_limit = args.episode_limit
        self.batch_size = args.batch_size
        self.episode_num = 0
        self.max_episode_len = 0
        self.buffer = None
        self.reset_buffer()


    def reset_buffer(self):
        self.buffer = {'s': np.zeros([self.batch_size, self.episode_limit, self.state_dim]),
                       'v': np.zeros([self.batch_size, self.episode_limit + 1]),
                       'a': np.zeros([self.batch_size, self.episode_limit]),
                       'a_logprob': np.zeros([self.batch_size, self.episode_limit]),
                       'r': np.zeros([self.batch_size, self.episode_limit]),
                       'dw': np.ones([self.batch_size, self.episode_limit]),  # Note: We use 'np.ones' to initialize 'dw'
                       'active': np.zeros([self.batch_size, self.episode_limit])
                       }
        self.episode_num = 0
        self.max_episode_len = 0


    def store_transition(self, episode_step, s, v, a, a_logprob, r, dw):
        self.buffer['s'][self.episode_num][episode_step] = s
        self.buffer['v'][self.episode_num][episode_step] = v
        self.buffer['a'][self.episode_num][episode_step] = a
        self.buffer['a_logprob'][self.episode_num][episode_step] = a_logprob
        self.buffer['r'][self.episode_num][episode_step] = r
        self.buffer['dw'][self.episode_num][episode_step] = dw

        self.buffer['active'][self.episode_num][episode_step] = 1.0


    def store_last_value(self, episode_step, v):
        self.buffer['v'][self.episode_num][episode_step] = v
        self.episode_num += 1
        # Record max_episode_len
        if episode_step > self.max_episode_len:
            self.max_episode_len = episode_step


    def get_adv(self):
        # Calculate the advantage using GAE
        v = self.buffer['v'][:, :self.max_episode_len]
        v_next = self.buffer['v'][:, 1:self.max_episode_len + 1]
        r = self.buffer['r'][:, :self.max_episode_len]
        dw = self.buffer['dw'][:, :self.max_episode_len]
        active = self.buffer['active'][:, :self.max_episode_len]
        adv = np.zeros_like(r)  # adv.shape=(batch_size,max_episode_len)
        gae = 0
        with torch.no_grad():  # adv and v_target have no gradient
            # deltas.shape=(batch_size,max_episode_len)
            deltas = r + self.gamma * v_next * (1 - dw) - v
            for t in reversed(range(self.max_episode_len)):
                gae = deltas[:, t] + self.gamma * self.lamda * gae  # gae.shape=(batch_size)
                adv[:, t] = gae
            v_target = adv + v  # v_target.shape(batch_size,max_episode_len)
            if self.use_adv_norm:  # Trick 1:advantage normalization
                adv_copy = copy.deepcopy(adv)
                adv_copy[active == 0] = np.nan  # 忽略掉active=0的那些adv
                adv = ((adv - np.nanmean(adv_copy)) / (np.nanstd(adv_copy) + 1e-5))
        return adv, v_target


    def get_training_data(self):
        adv, v_target = self.get_adv()
        batch = {'s': torch.tensor(self.buffer['s'][:, :self.max_episode_len], dtype=torch.float32),
                 'a': torch.tensor(self.buffer['a'][:, :self.max_episode_len], dtype=torch.long),  # 动作a的类型必须是long
                 'a_logprob': torch.tensor(self.buffer['a_logprob'][:, :self.max_episode_len], dtype=torch.float32),
                 'active': torch.tensor(self.buffer['active'][:, :self.max_episode_len], dtype=torch.float32),
                 'adv': torch.tensor(adv, dtype=torch.float32),
                 'v_target': torch.tensor(v_target, dtype=torch.float32)}

        return batch

# 3. PPO

In [ ]:
# Trick 8: orthogonal initialization
def orthogonal_init(layer, gain=np.sqrt(2)):
    for name, param in layer.named_parameters():
        if 'bias' in name:
            nn.init.constant_(param, 0)
        elif 'weight' in name:
            nn.init.orthogonal_(param, gain=gain)

    return layer

In [ ]:
class Actor_Critic_RNN(nn.Module):
    def __init__(self, args):
        super(Actor_Critic_RNN, self).__init__()
        self.use_gru = args.use_gru
        self.transformer = args.transformer
        self.activate_func = [nn.ReLU(), nn.Tanh()][args.use_tanh]  # Trick10: use tanh
        self.actor_rnn_hidden = None
        self.actor_fc1 = nn.Linear(args.state_dim, args.hidden_dim)
        self.actor_fc2 = nn.Linear(args.hidden_dim, args.hidden_dim)
        if args.transformer:
            self.actor_trans_key = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.actor_trans_value = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.actor_trans_query = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.actor_attention = nn.MultiheadAttention(args.hidden_dim, 8).to(DEVICE)
        else:
            if args.use_gru:
                print("------use GRU------")
                self.actor_rnn = nn.GRU(args.hidden_dim, args.hidden_dim, batch_first=True)
            else:
                print("------use LSTM------")
                self.actor_rnn = nn.LSTM(args.hidden_dim, args.hidden_dim, batch_first=True)
        self.actor_fc3 = nn.Linear(args.hidden_dim, args.action_dim)

        self.critic_rnn_hidden = None
        self.critic_fc1 = nn.Linear(args.state_dim, args.hidden_dim)
        self.critic_fc2 = nn.Linear(args.hidden_dim, args.hidden_dim)
        
        if args.transformer:
            print("------use transformer------")
            self.critic_trans_key = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.critic_trans_value = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.critic_trans_query = nn.Linear(args.hidden_dim, args.hidden_dim).to(DEVICE)
            self.critic_attention = nn.MultiheadAttention(args.hidden_dim, 8).to(DEVICE)
        else:
            if args.use_gru:
                self.critic_rnn = nn.GRU(args.hidden_dim, args.hidden_dim, batch_first=True)
            else:
                self.critic_rnn = nn.LSTM(args.hidden_dim, args.hidden_dim,batch_first=True)
        self.critic_fc3 = nn.Linear(args.hidden_dim, 1)

        if args.use_orthogonal_init:
            print("------use orthogonal init------")
            orthogonal_init(self.actor_fc1)
            orthogonal_init(self.actor_fc2, gain=0.01)
            orthogonal_init(self.actor_fc3)
            orthogonal_init(self.critic_fc1)
            orthogonal_init(self.critic_fc2)
            orthogonal_init(self.critic_fc3)
            if args.transformer:
                orthogonal_init(self.actor_trans_key)
                orthogonal_init(self.actor_trans_value)
                orthogonal_init(self.actor_trans_query)
                orthogonal_init(self.actor_attention)
                orthogonal_init(self.critic_trans_key)
                orthogonal_init(self.critic_trans_value)
                orthogonal_init(self.critic_trans_query)
                orthogonal_init(self.critic_attention)
            else:
                orthogonal_init(self.actor_rnn)
                orthogonal_init(self.critic_rnn)


    def actor(self, s):
        s = s.to(DEVICE)
        s = self.actor_fc1(s)
        s = self.activate_func(s)
        s = self.actor_fc2(s)
        s = self.activate_func(s)
        if self.transformer:
            key = self.actor_trans_key(s)
            value = self.actor_trans_value(s)
            query = self.actor_trans_query(s)
            output, attention_weight = self.actor_attention(query, key, value)
        else:
            output, self.actor_rnn_hidden = self.actor_rnn(s, self.actor_rnn_hidden)
        logit = self.actor_fc3(output)
        return logit


    def critic(self, s):
        s = s.to(DEVICE)
        s = self.critic_fc1(s)
        s = self.activate_func(s)
        s = self.critic_fc2(s)
        s = self.activate_func(s)
        if self.transformer:
            key = self.critic_trans_key(s)
            value = self.critic_trans_value(s)
            query = self.critic_trans_query(s)
            output, attention_weight = self.critic_attention(query, key, value)
        else:
            output, self.critic_rnn_hidden = self.critic_rnn(s, self.critic_rnn_hidden)
        value = self.critic_fc3(output)
        return value

In [ ]:
class PPO_discrete_RNN:
    def __init__(self, args):
        self.batch_size = args.batch_size
        self.mini_batch_size = args.mini_batch_size
        self.max_train_steps = args.max_train_steps
        self.lr = args.lr  # Learning rate of actor
        self.gamma = args.gamma  # Discount factor
        self.lamda = args.lamda  # GAE parameter
        self.epsilon = args.epsilon  # PPO clip parameter
        self.K_epochs = args.K_epochs  # PPO parameter
        self.entropy_coef = args.entropy_coef  # Entropy coefficient
        self.set_adam_eps = args.set_adam_eps
        self.use_grad_clip = args.use_grad_clip
        self.use_lr_decay = args.use_lr_decay
        self.use_adv_norm = args.use_adv_norm

        self.ac = Actor_Critic_RNN(args).cuda()
        if self.set_adam_eps:  # Trick 9: set Adam epsilon=1e-5
            self.optimizer = torch.optim.Adam(self.ac.parameters(), lr=self.lr, eps=1e-5)
        else:
            self.optimizer = torch.optim.Adam(self.ac.parameters(), lr=self.lr)

    def reset_rnn_hidden(self):
        self.ac.actor_rnn_hidden = None
        self.ac.critic_rnn_hidden = None

    def choose_action(self, s, evaluate=False):
        with torch.no_grad():
            s = torch.tensor(s, dtype=torch.float).unsqueeze(0)
            logit = self.ac.actor(s)
            if evaluate:
                a = torch.argmax(logit)
                return a.item(), None
            else:
                dist = Categorical(logits=logit)
                a = dist.sample()
                a_logprob = dist.log_prob(a)
                return a.item(), a_logprob.item()

    def get_value(self, s):
        with torch.no_grad():
            s = torch.tensor(s, dtype=torch.float).unsqueeze(0)
            value = self.ac.critic(s)
            return value.item()

    def train(self, replay_buffer, total_steps):
        batch = replay_buffer.get_training_data()  # Get training data

        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            for index in BatchSampler(SequentialSampler(range(self.batch_size)), self.mini_batch_size, False):
                # If use RNN, we need to reset the rnn_hidden of the actor and critic.
                self.reset_rnn_hidden()
                logits_now = self.ac.actor(batch['s'][index])  # logits_now.shape=(mini_batch_size, max_episode_len, action_dim)
                values_now = self.ac.critic(batch['s'][index]).squeeze(-1)  # values_now.shape=(mini_batch_size, max_episode_len)

                dist_now = Categorical(logits=logits_now)
                dist_entropy = dist_now.entropy()  # shape(mini_batch_size, max_episode_len)
                a_logprob_now = dist_now.log_prob(batch['a'][index])  # shape(mini_batch_size, max_episode_len)
                # a/b=exp(log(a)-log(b))
                ratios = torch.exp(a_logprob_now - batch['a_logprob'][index])  # shape(mini_batch_size, max_episode_len)

                # actor loss
                surr1 = ratios * batch['adv'][index]
                surr2 = torch.clamp(ratios, 1 - self.epsilon, 1 + self.epsilon) * batch['adv'][index]
                actor_loss = -torch.min(surr1, surr2) - self.entropy_coef * dist_entropy  # shape(mini_batch_size, max_episode_len)
                actor_loss = (actor_loss * batch['active'][index]).sum() / batch['active'][index].sum()

                # critic_loss
                critic_loss = (values_now - batch['v_target'][index]) ** 2
                critic_loss = (critic_loss * batch['active'][index]).sum() / batch['active'][index].sum()

                # Update
                self.optimizer.zero_grad()
                loss = actor_loss + critic_loss * 0.5
                loss.backward()
                if self.use_grad_clip:  # Trick 7: Gradient clip
                    torch.nn.utils.clip_grad_norm_(self.ac.parameters(), 0.5)
                self.optimizer.step()

        if self.use_lr_decay:  # Trick 6:learning rate Decay
            self.lr_decay(total_steps)
    
    def lr_decay(self, total_steps):
        lr_now = 0.9 * self.lr * (1 - total_steps / self.max_train_steps) + 0.1 * self.lr
        for p in self.optimizer.param_groups:
            p['lr'] = lr_now

    def save_model(self, env_name, number, total_steps):
        torch.save(self.ac.state_dict(), "model/PPO_actor_env_{}_number_{}_step_{}k.pth".format(env_name, number, total_steps))

    def load_model(self, env_name, number,step_num=None):
        # extract 
        if step_num >0:
            step = step_num
        else:
            step = max([int(x.split("_")[-1][:-5]) for x in os.listdir("model")])
        # step = "final"
        print("LOADING model/PPO_actor_env_{}_number_{}_step_{}k.pth".format(env_name, number, step))
        self.ac.load_state_dict(torch.load("model/PPO_actor_env_{}_number_{}_step_{}k.pth".format(env_name, number, step), map_location=DEVICE))

# 4. TRAIN

In [ ]:
class Runner:
    def __init__(self, args, env_name, number, seed):
        self.args = args
        self.env_name = env_name
        self.number = number
        self.seed = seed
        self.dataset = None
        self.save_model_freq = self.args.save_model_freq
        self.save_counter = 0
        self.last_save_counter=0
        self.load_last_weights = self.args.resume
        self.set_dataset()

        # Create env
        self.env = gym.make(env_name,dataset=self.dataset,num_agent=1)
        # Set random seed
        np.random.seed(self.seed)
        torch.manual_seed(self.seed)
        # self.env.seed(seed)
        # self.env.action_space.seed(seed)

        self.args.state_dim = self.env.observation_space.shape[0]
        self.args.action_dim = self.env.action_space.n
        self.args.episode_limit = self.env.max_steps_per_episode  # Maximum number of steps per episode
        print("env={}".format(env_name))
        print("state_dim={}".format(args.state_dim))
        print("action_dim={}".format(args.action_dim))
        print("episode_limit={}".format(args.episode_limit))

        self.replay_buffer = ReplayBuffer(args)
        self.agent = PPO_discrete_RNN(args)
        # Create a tensorboard

        self.evaluate_rewards = []  # Record the rewards during the evaluating
        self.total_steps = 0
        if self.load_last_weights:
            print("------Resuming train from step #",self.args.steps_num,"------")
            self.agent.load_model("VisualGrounding-v0", self.number,self.args.steps_num)
            self.total_steps = self.args.steps_num

        if self.args.use_state_norm:
            print("------use state normalization------")
            self.state_norm = Normalization(shape=args.state_dim)  # Trick 2:state normalization
        if self.args.use_reward_scaling:
            print("------use reward scaling------")
            self.reward_scaling = RewardScaling(shape=1, gamma=self.args.gamma)
        print("RUNNER INITIALIZED")


    def set_dataset(self):
        self.dataset = RefCOCOg(data_dir="../")


    def run(self, ):
        evaluate_num = -1  # Record the number of evaluations
        if self.load_last_weights:
            evaluate_num = self.total_steps // self.args.evaluate_freq
        while self.total_steps < self.args.max_train_steps:
            print("tot:{}, evaluate_num:{} , freq: {}".format(self.total_steps, evaluate_num, self.args.evaluate_freq))
            if self.total_steps // self.args.evaluate_freq > evaluate_num:
                self.evaluate_policy()  # Evaluate the policy every 'evaluate_freq' steps
                evaluate_num += 1
            _, episode_steps = self.run_episode()  # Run an episode
            self.total_steps += episode_steps

            if self.replay_buffer.episode_num == self.args.batch_size:
                self.agent.train(self.replay_buffer, self.total_steps)  # Training
                self.replay_buffer.reset_buffer()

        self.evaluate_policy()
        self.env.close()


    def run_episode(self, ):
        episode_reward = 0
        s,info = self.env.reset(options={"split":"train"})
        if self.args.use_reward_scaling:
            self.reward_scaling.reset()
        self.agent.reset_rnn_hidden()
        for episode_step in range(self.args.episode_limit):
            if self.args.use_state_norm:
                s = self.state_norm(s)
            a, a_logprob = self.agent.choose_action(s, evaluate=False)
            v = self.agent.get_value(s)
            s_, r, done, info = self.env.step(a)
            episode_reward += r

            if done and episode_step + 1 != self.args.episode_limit:
                dw = True
            else:
                dw = False
            if self.args.use_reward_scaling:
                r = self.reward_scaling(r)
            # Store the transition
            self.replay_buffer.store_transition(episode_step, s, v, a, a_logprob, r, dw)
            s = s_
            if done==True:
                break

        # An episode is over, store v in the last step
        if self.args.use_state_norm:
            s = self.state_norm(s)
        v = self.agent.get_value(s)
        self.replay_buffer.store_last_value(episode_step + 1, v)
        
        
        return episode_reward, episode_step + 1

    
    def evaluate_policy(self, ):
        print("EVALUATION START")
        evaluate_reward = 0
        evaluate_iou = 0
        iou_step = 0
        for _ in range(self.args.evaluate_times):
            episode_reward, done, info = 0, False, {}
            s, info = self.env.reset(options={"split":"val"})
            counter = 0
            self.agent.reset_rnn_hidden()
            while done==False and counter<self.args.episode_limit :
                if self.args.use_state_norm:
                    s = self.state_norm(s, update=False)
                a, a_logprob = self.agent.choose_action(s, evaluate=True)
                s_, r, done, info = self.env.step(a)
                episode_reward += r
                s = s_
                # print("trigger is ",done, " | #", counter," steps", counter<=self.args.episode_limit)
                counter+=1
            print("episode_reward: ",episode_reward, "\t|\t num_actions: ",counter)
            evaluate_reward += episode_reward
            iou = torchvision.ops.box_iou(torch.tensor(info["target_bbox"]).to(DEVICE),torch.tensor(info["pred_bbox"]).to(DEVICE)).item()
            evaluate_iou += iou
            iou_step += 1
        evaluate_reward = evaluate_reward / self.args.evaluate_times
        self.evaluate_rewards.append(evaluate_reward)
        print("total_steps:{} \t mean_iou: {}evaluate_reward:{}".format(self.total_steps,evaluate_iou/iou_step, evaluate_reward))
        # Save the rewards and models
        wandb.log( { "mean_iou":evaluate_iou/iou_step, "mean_reward": evaluate_reward},step=self.total_steps )
        if self.total_steps - self.last_save_counter > self.save_model_freq:
            print("SAVING MODEL")
            self.agent.save_model(self.env_name, self.number, self.total_steps)
            self.last_save_counter = self.total_steps
        print("Total steps: ",self.total_steps)
        print("EVALUATION END")

### TODO: sistemare parser

In [ ]:
torch.set_default_device("cuda:0")
gym.envs.register(
    id='VisualGrounding-v0',
    entry_point='env.env:VisualGroundingEnv'
)

parser = argparse.ArgumentParser("Hyperparameter Setting for PPO-discrete")
parser.add_argument("-n", type=int, default=int(-1), help=" name index")
parser.add_argument("--max_train_steps", type=int, default=int(2e6), help=" Maximum number of training steps")
parser.add_argument("--evaluate_freq", type=float, default=5e3, help="Evaluate the policy every 'evaluate_freq' steps")
parser.add_argument("--save_model_freq", type=int, default=2e4, help="Save frequency")
parser.add_argument("--evaluate_times", type=float, default=100, help="Evaluate times")

parser.add_argument("--batch_size", type=int, default=1024, help="Batch size")
parser.add_argument("--mini_batch_size", type=int, default=128, help="Minibatch size")
parser.add_argument("--hidden_dim", type=int, default=1024, help="The number of neurons in hidden layers of the neural network")
parser.add_argument("--lr", type=float, default=3e-4, help="Learning rate of actor")
parser.add_argument("--gamma", type=float, default=0.99, help="Discount factor")
parser.add_argument("--lamda", type=float, default=0.95, help="GAE parameter")
parser.add_argument("--epsilon", type=float, default=0.20, help="PPO clip parameter")
parser.add_argument("--K_epochs", type=int, default=15, help="PPO parameter")
parser.add_argument("--use_adv_norm", type=bool, default=True, help="Trick 1:advantage normalization")
parser.add_argument("--use_state_norm", type=bool, default=False, help="Trick 2:state normalization")
parser.add_argument("--use_reward_scaling", type=bool, default=True, help="Trick 4:reward scaling")
parser.add_argument("--entropy_coef", type=float, default=0.01, help="Trick 5: policy entropy")
parser.add_argument("--use_lr_decay", type=bool, default=True, help="Trick 6:learning rate Decay")
parser.add_argument("--use_grad_clip", type=bool, default=True, help="Trick 7: Gradient clip")
parser.add_argument("--use_orthogonal_init", type=bool, default=True, help="Trick 8: orthogonal initialization")
parser.add_argument("--set_adam_eps", type=float, default=True, help="Trick 9: set Adam epsilon=1e-5")
parser.add_argument("--use_tanh", type=float, default=True, help="Trick 10: tanh activation function")
parser.add_argument("--use_gru", type=bool, default=False, help="Whether to use GRU")
parser.add_argument("--resume", type=bool, default=False, help="load last weights and resume training from checkpoint")
parser.add_argument("--steps_num", type=int, default=-1, help="steps number to load weights")
parser.add_argument("--transformer", type=bool, default=False, help="whether to use transformer instead of lstm")

# args = parser.parse_args()
# Jupyter Notebook argparse compatibility test
args = argparser.Namespace(
    n = int(-1),
    max_train_steps = int(2e6),
    evaluate_freq = 5e3,
    save_model_freq = 2e4,
    evaluate_times = 100,
    batch_size = 1024,
    mini_batch_size = 128,
    hidden_dim = 1024,
    lr = 3e-4,
    gamma = 0.99,
    lamda = 0.95,
    epsilon = 0.20,
    K_epochs = 15,
    use_adv_norm = True,
    use_state_norm = False,
    use_reward_scaling = True,
    entropy_coef = 0.01,
    use_lr_decay = True,
    use_grad_clip = True,
    use_orthogonal_init = True,
    set_adam_eps = True,
    use_tanh = True,
    use_gru = False,
    resume = False,
    steps_num = -1,
    transformer = False,
    )
print(args)

if args.n == -1:
    print("Please input the index of the model")
    exit()
if args.steps_num == -1 and args.resume==True:
    print("Please input the steps number of the checkpoint")
    exit()
run = wandb.init(project="lstm-RL-PPO",id="12eps+LSTM+Trigger-Fixed+YOLOProp",resume=args.resume, config={
    "lstm_hidden_size":1024,
    "batch_size":1024,
    "lr":3e-4,
    "gamma":0.99,
    "mini_batch_size":64,
    "episode_limit":12,
    "use_history":True,
    "epsilon":0.2,
    "embedding_fusion":False,
    "convergence_threshold":0.5
})
env_names = ['VisualGrounding-v0']
env_index = 0
for seed in [0, 10, 100]:
    runner = Runner(args, env_name=env_names[env_index], number=args.n, seed=seed)
    runner.run()

# 5. TEST

In [ ]:
def denormalize(img,x0_norm,y0_norm,x1_norm,y1_norm):
    width = img.shape[1]
    height = img.shape[0]
    # print("den",width,height,sep=" | ")
    x0 = int(x0_norm * width)
    y0 = int(y0_norm * height)
    x1 = int(x1_norm * width)
    y1 = int(y1_norm * height)
    return x0,y0,x1,y1

In [ ]:
def compute_iou(agent, ground_truth):
    iou =  torchvision.ops.box_iou( agent, ground_truth)[0].item()
    print("iou : ",iou)
    return iou

In [ ]:
action_dict = {
    0: 'right',
    1: 'left',
    2: 'up',
    3: 'down',
    4: 'taller',
    5: 'fatter',
    6: 'shorter',
    7: 'thinner',
    8: 'trigger'
}

In [ ]:
class Test():
    def __init__(self, args, number):
        self.args = args
        gym.envs.register(
            id='VisualGrounding-v0',
            entry_point='env.env:VisualGroundingEnv'
            )
        self.number = number #id of model
        self.dataset = RefCOCOg("../","val")
        self.env = gym.make("VisualGrounding-v0",dataset=self.dataset,num_agent=1,random_validation=False)
        self.args.state_dim = self.env.observation_space.shape[0]
        self.args.action_dim = self.env.action_space.n
        self.args.episode_limit = self.env.max_steps_per_episode
        self.agent = PPO_discrete_RNN(self.args)
        self.agent.load_model("VisualGrounding-v0", self.number,self.args.steps_num)
        self.state_norm = Normalization(shape=args.state_dim)  # Trick 2:state normalization


    def show_result(self,image,sentences,info):
        gt_x0,gt_y0,gt_x1,gt_y1 = int(info["target_bbox"][0][0].item()),int(info["target_bbox"][0][1].item()),int(info["target_bbox"][0][2].item()),int(info["target_bbox"][0][3].item())
        x0,y0,x1,y1 = int(info["pred_bbox"][0][0].item()),int(info["pred_bbox"][0][1].item()),int(info["pred_bbox"][0][2].item()),int(info["pred_bbox"][0][3].item())
        #draw predicted bbox
        bbox_img = cv2.rectangle(np.array(image), (int(x0), int(y0)), (int(x1), int(y1)), (255,0,0), 2)
        #draw ground truth
        bbox_img = cv2.rectangle(bbox_img,(int(gt_x0),int(gt_y0)),(int(gt_x1),int(gt_y1)), (0,0,255), 2)
        
        # bbox_img = cv2.resize(bbox_img, (int(bbox_img.shape[1]/3), int(bbox_img.shape[0]/3)))

        Image.fromarray(bbox_img).show()
        print("Groud Truth: {:>5} {:>5} {:>5} {:>5}  |    agent_location: {:>5} {:>5} {:>5} {:>5}".format(gt_x0,gt_y0,gt_x1,gt_y1,x0,y0,x1,y1))
        print("sentences:\n",sentences)
        tm.sleep(7)


    def evaluate(self):
        evaluate_reward = 0
        evaluate_iou, iou_count = 0, 0
        accuracy_count = 0
        for i in range(0,VALIDATION_SET_SIZE):
            episode_reward, done, info = 0, False, {}
            s, info = self.env.reset(options={"split":"val"})
            img_idx = info["img_idx"]
            _,_,_,_, image, sentences= self.dataset[img_idx,"val"]
            self.agent.reset_rnn_hidden()
            counter = 0
            actions=[]
            
            while info["trigger_pressed"]==False and  counter < 12:
                if self.args.use_state_norm:
                    s = self.state_norm(s)
                a, a_logprob = self.agent.choose_action(s, evaluate=True)
                s_, r, done, info = self.env.step(a)
                print("received reward: ",r)
                episode_reward += r
                s = s_
                counter+=1
                # print("Episode reward: ",episode_reward)
                actions.append(action_dict[a])
                if a == 8:
                    print("Trigger pressed after {} steps".format(counter))
                    # self.show_result(image,sentences,info)
            print("Episode reward: ",episode_reward)
            iou = torchvision.ops.box_iou(info["target_bbox"],info["pred_bbox"]).item()
            iou_count += 1
            if iou > 0.5:
                accuracy_count += 1
            evaluate_iou += iou
            print("#{: >5} ->BBOX_IOU: {: >10} | mean IOU: {: >10} | accuracy: {: >10}".format(i,round(iou, 4),round(evaluate_iou/iou_count,4),round(accuracy_count/iou_count,4)))
            print("Actions: ",actions)
            print("\n\n#######################################################################")
            evaluate_reward += episode_reward

### TODO: STESSA COSA DI PRIMA

In [ ]:
parser = argparse.ArgumentParser("Hyperparameter Setting for PPO-discrete")
parser.add_argument("-n", type=int, default=int(-1), help=" name index")
parser.add_argument("--max_train_steps", type=int, default=int(2e6), help=" Maximum number of training steps")
parser.add_argument("--evaluate_freq", type=float, default=5e3, help="Evaluate the policy every 'evaluate_freq' steps")
parser.add_argument("--save_model_freq", type=int, default=2e4, help="Save frequency")
parser.add_argument("--evaluate_times", type=float, default=100, help="Evaluate times")

parser.add_argument("--batch_size", type=int, default=1024, help="Batch size")
parser.add_argument("--mini_batch_size", type=int, default=128, help="Minibatch size")
parser.add_argument("--hidden_dim", type=int, default=1024, help="The number of neurons in hidden layers of the neural network")
parser.add_argument("--lr", type=float, default=3e-4, help="Learning rate of actor")
parser.add_argument("--gamma", type=float, default=0.99, help="Discount factor")
parser.add_argument("--lamda", type=float, default=0.95, help="GAE parameter")
parser.add_argument("--epsilon", type=float, default=0.20, help="PPO clip parameter")
parser.add_argument("--K_epochs", type=int, default=15, help="PPO parameter")
parser.add_argument("--use_adv_norm", type=bool, default=True, help="Trick 1:advantage normalization")
parser.add_argument("--use_state_norm", type=bool, default=False, help="Trick 2:state normalization")
parser.add_argument("--use_reward_scaling", type=bool, default=True, help="Trick 4:reward scaling")
parser.add_argument("--entropy_coef", type=float, default=0.01, help="Trick 5: policy entropy")
parser.add_argument("--use_lr_decay", type=bool, default=True, help="Trick 6:learning rate Decay")
parser.add_argument("--use_grad_clip", type=bool, default=True, help="Trick 7: Gradient clip")
parser.add_argument("--use_orthogonal_init", type=bool, default=True, help="Trick 8: orthogonal initialization")
parser.add_argument("--set_adam_eps", type=float, default=True, help="Trick 9: set Adam epsilon=1e-5")
parser.add_argument("--use_tanh", type=float, default=True, help="Trick 10: tanh activation function")
parser.add_argument("--use_gru", type=bool, default=False, help="Whether to use GRU")
parser.add_argument("--resume", type=bool, default=False, help="load last weights and resume training from checkpoint")
parser.add_argument("--steps_num", type=int, default=-1, help="steps number to load weights")
parser.add_argument("--transformer", type=bool, default=False, help="whether to use transformer instead of lstm")
args = parser.parse_args()
if args.n == -1:
    print("Please input the index of the model")
    exit()
test = Test(args,args.n)
test.evaluate()